In [8]:
# imports
import os
import shutil
from pathlib import Path
import urllib.request
import pandas as pd
from google.cloud import bigquery
from prefect import task, flow
from prefect_gcp.cloud_storage import GcsBucket

print("Setup Complete")


# Deployment 1
# Fetch the data from Github url
@task(log_prints=True, name="fetch-file-dataset_url")
def fetch(dataset_url: str):
    filename, _ = urllib.request.urlretrieve(dataset_url)
    return filename


# Read and tweak to fix the dtypes of pick-up and drop-off
@task(log_prints=True, name="read-and-tweak-df")
def read_tweak_df(src: str) -> pd.DataFrame:
    df = pd.read_csv(src, parse_dates=[2, 3], compression="gzip")
    return df


# Write DataFrame to a specific folder after tweaking the DataFrame
@task(log_prints=True, name="write-to-local-file")
def write_local(df: pd.DataFrame, year: int, dataset_file: str) -> Path:
    directory = Path(f"{year}")
    path_name = directory / f"{dataset_file}.csv.gz"
    try:
        os.makedirs(directory, exist_ok=True)
        df.to_csv(path_name, compression="gzip")
    except OSError as error:
        print(error)
    return path_name


# https://app.prefect.cloud/account/975bd9ed-5aef-4c8a-a413-23073fef3acb/workspace/a711abba-f5ae-4b00-9315-34f92f089b77/blocks/catalog
# Upload local csv.gz file to GCS
@task(log_prints=True)
def write_gcs(path: Path) -> None:
    gcs_block = GcsBucket.load("prefect-gcs-block-ny-taxi")
    gcs_block.upload_from_path(from_path=path, to_path=path)
    print("Loaded data to GCS...Hooray!")
    return


# Delete file after uploads
@task(log_prints=True, name="dedeplicate-local-data")
def deduplicate(path: Path) -> None:
    try:
        os.remove(path)
        os.rmdir(path)
        print("Successfully deleted directory and local files...hep hep hooray")
    except OSError as error:
        print(f"The system cannot find the file specified: {error}")
    return


@flow(log_prints=True, name="etl-web-gcs")
def etl_web_gcs(year: int, month: int):
    # Parameters
    year = 2019
    month = 5
    dataset_file = f"fhv_tripdata_{year}-{month:02}"
    dataset_url = f"https://github.com/DataTalksClub/nyc-tlc-data/releases/download/fhv/{dataset_file}.csv.gz"

    # Execution
    # Fetch the data from Github url
    source = fetch(dataset_url)

    # no choice but to tweak the data types for us to upload this to BQ
    # Read and tweak to fix the dytpes of pick-up and drop-off
    df_tweak = read_tweak_df(source)

    # write df to local
    path_local = write_local(df_tweak, year, dataset_file)

    # Upload dataset from local to gcs
    write_gcs(path_local)

    # Delete file after uploads
    deduplicate(path_local)

    # Next step is to create a separate deployment from GCS to BigQuery


# Create Parent flow to loop
@flow(name="etl-parent-local-gcs")
def etl_parent_web_gcs(year: int, months: list):
    year = 2019
    months = [2]

    for month in months:
        etl_web_gcs(year, month)


# Run main
if __name__ == "__main__":
    etl_parent_web_gcs(year=2019, months=[2])


Setup Complete


/opt/homebrew/Caskroom/miniforge/base/lib/python3.9/site-packages/prefect/tasks.py:261: UserWarning: A task named 'fetch-file-dataset_url' and defined at '/var/folders/h2/6110hgtx6b7400ybw7yjhn400000gn/T/ipykernel_21033/1031823537.py:16' conflicts with another task. Consider specifying a unique `name` parameter in the task definition:

 `@task(name='my_unique_name', ...)`
  warnings.warn(
/opt/homebrew/Caskroom/miniforge/base/lib/python3.9/site-packages/prefect/tasks.py:261: UserWarning: A task named 'read-and-tweak-df' and defined at '/var/folders/h2/6110hgtx6b7400ybw7yjhn400000gn/T/ipykernel_21033/1031823537.py:23' conflicts with another task. Consider specifying a unique `name` parameter in the task definition:

 `@task(name='my_unique_name', ...)`
  warnings.warn(
/opt/homebrew/Caskroom/miniforge/base/lib/python3.9/site-packages/prefect/tasks.py:261: UserWarning: A task named 'write-to-local-file' and defined at '/var/folders/h2/6110hgtx6b7400ybw7yjhn400000gn/T/ipykernel_21033/1031

12:01:21.462 | INFO    | prefect.engine - Created flow run 'idealistic-puma' for flow 'etl-parent-local-gcs'

/opt/homebrew/Caskroom/miniforge/base/lib/python3.9/site-packages/prefect/tasks.py:261: UserWarning: A task named 'etl-web-gcs' and defined at '/var/folders/h2/6110hgtx6b7400ybw7yjhn400000gn/T/ipykernel_21033/1031823537.py:63' conflicts with another task. Consider specifying a unique `name` parameter in the task definition:

 `@task(name='my_unique_name', ...)`
  warnings.warn(


12:01:27.995 | INFO    | Flow run 'idealistic-puma' - Created subflow run 'towering-bison' for flow 'etl-web-gcs'

...

12:02:14.845 | INFO    | Flow run 'towering-bison' - Executing 'dedeplicate-local-data-8266ec2d-0' immediately...

12:02:15.884 | INFO    | Task run 'dedeplicate-local-data-8266ec2d-0' - The system cannot find the file specified: [Errno 2] No such file or directory: '~/2019/fhv_tripdata_2019-02.csv.gz'

12:02:16.215 | INFO    | Task run 'dedeplicate-local-data-8266ec2d-0' - Finished in state Completed()

12:02:16.593 | INFO    | Flow run 'towering-bison' - Finished in state Completed('All states completed.')

12:02:17.741 | INFO    | Flow run 'idealistic-puma' - Finished in state Completed('All states completed.')